# CS5785 final
## Xianhui Li, Zimeng Zhu

In [11]:
import os
import csv
import random
import gensim
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
import string

num_train = 8000
num_dev = 2000
num_test = 2000
split_idx = list(range(num_train + num_dev))
random.shuffle(split_idx)
COMPONMENT=100
ITER=100

In [2]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

# build x matrices
train_dev_desc = parse_descriptions("descriptions_train", num_doc=(num_train+num_dev))
test_desc = parse_descriptions("descriptions_test", num_doc=num_test)
x_train = np.array([train_dev_desc[i] for i in split_idx[:num_train]])
x_dev = np.array([train_dev_desc[i] for i in split_idx[num_train:]])
x_test = np.array([d for d in test_desc])

In [3]:
def parse_tags(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

# build x matrices
train_dev_tag = parse_tags("tags_train", num_doc=(num_train+num_dev))
test_tag = parse_tags("tags_test", num_doc=num_test)
z_train = np.array([train_dev_tag[i] for i in split_idx[:num_train]])
z_dev = np.array([train_dev_tag[i] for i in split_idx[num_train:]])
z_test = np.array([d for d in test_tag])

In [4]:
## Preprocess x train dataset
new_x_train=[]
for i in x_train:
    ## Lowercase all of the words
    i = i.lower()
    
    ## strip punctuation
    i = i.translate(str.maketrans('','',string.punctuation))
    i = i.translate(str.maketrans('','','1234567890'))
    
    ## strip stop words
    parse = i.split()
    parse = [word for word in parse if word not in stopwords.words('english')]
    
    ## Lemmatization of all the words
    lmtzr = WordNetLemmatizer()
    parse = [lmtzr.lemmatize(a) for a in parse]
    
    new_x_train.append(parse)

In [5]:
## Preprocess x_test dataset
new_x_test=[]
for i in x_test:
    ## Lowercase all of the words
    i = i.lower()
    
    ## strip punctuation
    i = i.translate(str.maketrans('','',string.punctuation))
    i = i.translate(str.maketrans('','','1234567890'))
    
    ## strip stop words
    parse = i.split()
    parse = [word for word in parse if word not in stopwords.words('english')]
    
    ## Lemmatization of all the words
    lmtzr = WordNetLemmatizer()
    parse = [lmtzr.lemmatize(a) for a in parse]
    ##st = LancasterStemmer()
    ##parse = [st.stem(a) for a in parse]
    
    new_x_test.append(parse)

In [6]:
## Preprocess x_dev dataset
new_x_dev=[]
for i in x_dev:
    ## Lowercase all of the words
    i = i.lower()
    
    ## strip punctuation
    i = i.translate(str.maketrans('','',string.punctuation))
    i = i.translate(str.maketrans('','','1234567890'))
    
    ## strip stop words
    parse = i.split()
    parse = [word for word in parse if word not in stopwords.words('english')]
    
    ## Lemmatization of all the words
    lmtzr = WordNetLemmatizer()
    parse = [lmtzr.lemmatize(a) for a in parse]
    ##st = LancasterStemmer()
    ##parse = [st.stem(a) for a in parse]
    
    new_x_dev.append(parse)

In [7]:
def change_to_01(x):
    for i in range(len(x)):
        for j in range(len(x[0])):
            x[i][j] = min(1, x[i][j]) 
    return x

In [8]:
vectorizer = CountVectorizer()
x_train = []
for i in range(len(new_x_train)):
    x_train.append(' '.join(new_x_train[i]))

x_train_bow = vectorizer.fit_transform(x_train).toarray()
x_train_bow = change_to_01(x_train_bow)
vocab = np.array(vectorizer.get_feature_names())

x_test = []
for i in range(len(new_x_test)):
    x_test.append(' '.join(new_x_test[i]))
    
vectorizer2 = CountVectorizer(vocabulary = vocab)
x_test_bow = vectorizer2.fit_transform(x_test)
x_test_bow = x_test_bow.toarray()
x_test_bow = change_to_01(x_test_bow)

x_dev = []
for i in range(len(new_x_dev)):
    x_dev.append(' '.join(new_x_dev[i]))

x_dev_bow = vectorizer2.fit_transform(x_dev).toarray()
x_dev_bow = change_to_01(x_dev_bow)

In [9]:
vectorizer_z = CountVectorizer()
z_train_bow = vectorizer_z.fit_transform(z_train).toarray()
z_train_bow = change_to_01(z_train_bow)
vocab = np.array(vectorizer_z.get_feature_names())
#z_train_tfidf = transformer.fit_transform(z_train_bow).toarray()

vectorizer_z2 = CountVectorizer(vocabulary = vocab)
z_test_bow = vectorizer_z2.fit_transform(z_test).toarray()
z_test_bow = change_to_01(z_test_bow)
#z_test_tfidf = transformer.fit_transform(z_test_bow).toarray()

z_dev_bow = vectorizer_z2.fit_transform(z_dev).toarray()
z_dev_bow = change_to_01(z_dev_bow)
#z_dev_tfidf = transformer.fit_transform(z_dev_bow)

In [12]:
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import GridSearchCV
from sklearn.cross_decomposition import PLSRegression

# train OLS model with regression
# parameters = {"alpha": [0.5, 1.0, 3.0]}
# reg = GridSearchCV(Ridge(), parameters)
pls = PLSRegression(n_components=COMPONMENT, max_iter=ITER)
pls.fit(z_train_bow, x_train_bow)

/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')
/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:313: UserWarning: X scores are null at iteration 92
  warnings.warn('X scores are null at iteration %s' % k)


PLSRegression(copy=True, max_iter=100, n_components=100, scale=True,
       tol=1e-06)

In [13]:
import sklearn.metrics
def dist_matrix(x1, x2):
    return sklearn.metrics.pairwise.cosine_distances(x1, x2)
#     return ((np.expand_dims(x1, 1) - np.expand_dims(x2, 0)) ** 2).sum(2) ** 0.5

# test performance on development set
x_dev_pred = pls.predict(z_dev_bow)
dev_distances = dist_matrix(x_dev_bow, x_dev_pred)
dev_scores = []
dev_pos_list = []

for i in range(num_dev):
    pred_dist_idx = list(np.argsort(dev_distances[i]))
    dev_pos = pred_dist_idx.index(i)
    dev_pos_list.append(dev_pos)
    if dev_pos < 20:
        dev_scores.append(1 / (dev_pos + 1))
    else:
        dev_scores.append(0.0)

print("Development MAP@20:", np.mean(dev_scores))
print("Mean index of true image", np.mean(dev_pos_list))
print("Median index of true image", np.median(dev_pos_list))

Development MAP@20: 0.2563552649534847
Mean index of true image 18.0385
Median index of true image 8.0


In [14]:
# create test predictions
x_train_all = np.concatenate([x_train_bow, x_dev_bow])
z_train_all = np.concatenate([z_train_bow, z_dev_bow])

#pls2 = PLSRegression(n_components=COMPONMENT, max_iter=ITER)
pls.fit(z_train_all, x_train_all)
x_test_pred = pls.predict(z_test_bow)


/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')
/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:313: UserWarning: X scores are null at iteration 92
  warnings.warn('X scores are null at iteration %s' % k)


In [15]:
test_distances = dist_matrix(x_test_bow, x_test_pred)
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("xianhui.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!
